In [1]:
import os
import re

import datetime

import pandas as pd
import dask.dataframe as dd

import zipfile
import plotly.express as px

from IPython.display import clear_output

In [2]:
#path_data = "D:/CSUN/COMED/Comed DATA/201801.zip" 
#zip = zipfile.ZipFile(path_data)
#files= (zip.namelist())
month = '201711'
zips = [os.path.join(dirpath,f) for (dirpath, dirnames, filenames) in os.walk(f'data/comed/{month}/') for f in filenames]
zips = zips
N_zips = len(zips)

In [3]:
def get_HR(x):
    time_str = re.search(r'INTERVAL_HR(.*?)_ENERGY_QTY', x)[1]
    return time_str

selected_columns = ['ZIP_CODE', 'INTERVAL_READING_DATE', 
                    'ACCOUNT_IDENTIFIER',
                    'INTERVAL_HR0030_ENERGY_QTY', 
                    'INTERVAL_HR0100_ENERGY_QTY', 'INTERVAL_HR0130_ENERGY_QTY',
                    'INTERVAL_HR0200_ENERGY_QTY', 'INTERVAL_HR0230_ENERGY_QTY',
                    'INTERVAL_HR0300_ENERGY_QTY', 'INTERVAL_HR0330_ENERGY_QTY',
                    'INTERVAL_HR0400_ENERGY_QTY', 'INTERVAL_HR0430_ENERGY_QTY',
                    'INTERVAL_HR0500_ENERGY_QTY', 'INTERVAL_HR0530_ENERGY_QTY',
                    'INTERVAL_HR0600_ENERGY_QTY', 'INTERVAL_HR0630_ENERGY_QTY',
                    'INTERVAL_HR0700_ENERGY_QTY', 'INTERVAL_HR0730_ENERGY_QTY',
                    'INTERVAL_HR0800_ENERGY_QTY', 'INTERVAL_HR0830_ENERGY_QTY',
                    'INTERVAL_HR0900_ENERGY_QTY', 'INTERVAL_HR0930_ENERGY_QTY',
                    'INTERVAL_HR1000_ENERGY_QTY', 'INTERVAL_HR1030_ENERGY_QTY',
                    'INTERVAL_HR1100_ENERGY_QTY', 'INTERVAL_HR1130_ENERGY_QTY',
                    'INTERVAL_HR1200_ENERGY_QTY', 'INTERVAL_HR1230_ENERGY_QTY',
                    'INTERVAL_HR1300_ENERGY_QTY', 'INTERVAL_HR1330_ENERGY_QTY',
                    'INTERVAL_HR1400_ENERGY_QTY', 'INTERVAL_HR1430_ENERGY_QTY',
                    'INTERVAL_HR1500_ENERGY_QTY', 'INTERVAL_HR1530_ENERGY_QTY',
                    'INTERVAL_HR1600_ENERGY_QTY', 'INTERVAL_HR1630_ENERGY_QTY',
                    'INTERVAL_HR1700_ENERGY_QTY', 'INTERVAL_HR1730_ENERGY_QTY',
                    'INTERVAL_HR1800_ENERGY_QTY', 'INTERVAL_HR1830_ENERGY_QTY',
                    'INTERVAL_HR1900_ENERGY_QTY', 'INTERVAL_HR1930_ENERGY_QTY',
                    'INTERVAL_HR2000_ENERGY_QTY', 'INTERVAL_HR2030_ENERGY_QTY',
                    'INTERVAL_HR2100_ENERGY_QTY', 'INTERVAL_HR2130_ENERGY_QTY',
                    'INTERVAL_HR2200_ENERGY_QTY', 'INTERVAL_HR2230_ENERGY_QTY',
                    'INTERVAL_HR2300_ENERGY_QTY', 'INTERVAL_HR2330_ENERGY_QTY',
                    'INTERVAL_HR0000_ENERGY_QTY']

In [4]:
dict_col_agg = {}

for col in selected_columns[2:]:
    dict_col_agg[col] = 'sum'
#dict_col_agg['DELIVERY_SERVICE_NAME'] = lambda x: (x.value_counts().index, x.value_counts())
#dict_col_agg['DELIVERY_SERVICE_CLASS'] = lambda x: (x.value_counts().index, x.value_counts())
dict_col_agg['ACCOUNT_IDENTIFIER'] = 'count'

In [20]:
months = os.listdir('data/comed/')

for month in months[:-1]:
    zips = [os.path.join(dirpath,f) for (dirpath, dirnames, filenames) in os.walk(f'data/comed/{month}/') for f in filenames]
    zips = zips
    N_zips = len(zips)

    df_total = pd.DataFrame()
    #df_total = dd.from_pandas(df_total, npartitions=1)

    count = 0

    for zip_path in zips:
        zip = zipfile.ZipFile(zip_path)
        count = count + 1
        clear_output(wait=True)
        print(f'{count}/{N_zips} --------> {round(100*count/N_zips, 3)} % progress in month {month}')
        for file in zip.namelist():
            zip2 = zipfile.ZipFile(zip.open(file))
            df = pd.read_csv(zip2.open(zip2.namelist()[0]))
            df['ZIP_CODE'] = df['ZIP_CODE'].str[:5]
            df = df.rename(columns={'INTERVAL_HR2400_ENERGY_QTY': 'INTERVAL_HR0000_ENERGY_QTY'})
            df = df.groupby(['ZIP_CODE','INTERVAL_READING_DATE']).agg(dict_col_agg).reset_index()
            df_total = pd.concat([df_total,df])
            df_total = df_total.groupby(['ZIP_CODE', 'INTERVAL_READING_DATE']).sum().reset_index()
            #df_total = dd.concat([df_total, dd.from_pandas(pd.read_csv(zip2.open(zip2.namelist()[0])), npartitions=1)])


    df = df_total

    #df['DELIVERY_SERVICE_NAME'] = df.DELIVERY_SERVICE_NAME.astype(str)
    #df['DELIVERY_SERVICE_CLASS'] = df.DELIVERY_SERVICE_CLASS.astype(str)

    df = df.rename(columns={'INTERVAL_HR2400_ENERGY_QTY': 'INTERVAL_HR0000_ENERGY_QTY'})

    #df = df.drop(['DELIVERY_SERVICE_NAME', 'DELIVERY_SERVICE_CLASS'], axis=1)

    print('Reshaping data...')

    df = df[selected_columns].melt(
        id_vars=["ZIP_CODE", "ACCOUNT_IDENTIFIER", "INTERVAL_READING_DATE"],
        var_name="Time",
        value_name="energy")

  
    df['Time'] = df.Time.map(lambda x: get_HR(x))
    df['date_time'] = df.INTERVAL_READING_DATE  + " " + df.Time
    df['date_time'] = df.date_time.map(lambda x: datetime.datetime.strptime(x,'%m/%d/%Y %H%M')).astype(str)

    df = df[['ZIP_CODE', 'ACCOUNT_IDENTIFIER', 'date_time', 'energy']]

    df['zip5'] = df.ZIP_CODE.str[:5]

    df = df.rename(columns={'ZIP_CODE': 'zip9','ACCOUNT_IDENTIFIER': 'n_acc'})

    
    df = df[['zip5', 'n_acc', 'date_time', 'energy']]

    #df = df.repartition(500)

    print('Exporting data...')

    df.to_csv(f'data/comed_month/comed_{month}.csv', index=False)

    del df
    del df_total


51/51 --------> 100.0 % progress in month 201712


BadZipFile: File is not a zip file

In [32]:
fig = px.line(df.query('acc_id==@accounts').sort_values('date_time'), x="date_time", y="energy", color='acc_id')
fig.show()